In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.special import logit
pd.options.display.max_columns = 100

In [2]:
df = pd.read_csv('soccer18_full.csv')
df = df.rename(columns = {'HomeTeam':'Team_Home','AwayTeam':'Team_Away'})
df = df.sort_values(['Date','game_id'])
dfs = pd.read_csv('soccer18_shots.csv')

In [3]:
df.head()

,Div,Date,Y,Team_Home,Team_Away,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,pH,pD,pA,home_xG,away_xG,home_team_id,away_team_id,game_id,pOver,pUnder
5306,Ligue_1,2014-08-08,14,Reims,Paris SG,2,2,2,1,9,16,3,6,0.089841,0.196675,0.713484,1.367870,2.655380,177,161,6185,0.531328,0.468672
5307,Ligue_1,2014-08-09,14,Bastia,Marseille,3,3,1,2,13,9,4,4,0.245226,0.292215,0.462559,1.396890,1.317510,162,164,6186,0.433584,0.566416
5308,Ligue_1,2014-08-09,14,Evian Thonon Gaillard,Caen,0,3,0,3,10,12,2,7,0.399734,0.297363,0.302904,0.813737,1.238690,209,165,6187,0.409639,0.590361
5309,Ligue_1,2014-08-09,14,Guingamp,St Etienne,0,2,0,1,6,7,3,2,0.282701,0.312602,0.404697,0.632940,1.495320,169,175,6188,0.379147,0.620853
5310,Ligue_1,2014-08-09,14,Lille,Metz,0,0,0,0,14,2,3,1,0.636756,0.235893,0.127351,1.544680,0.057137,160,180,6189,0.428922,0.571078


In [4]:
dfs.head()

,shot_id,game_id,team_id,result,x,y,xG,situation,shot_type,player_assisted,last_action,minute,player
0,1,6185,161,MissedShots,0.898,0.707,0.048068,OpenPlay,LeftFoot,Zlatan Ibrahimovic,Pass,5,Edinson Cavani
1,2,6185,161,Goal,0.900,0.609,0.381072,OpenPlay,LeftFoot,Javier Pastore,Throughball,6,Zlatan Ibrahimovic
2,3,6185,177,BlockedShot,0.868,0.525,0.069345,OpenPlay,RightFoot,NaN,None,10,Gaëtan Charbonnier
3,4,6185,177,MissedShots,0.745,0.574,0.018125,OpenPlay,LeftFoot,Johny Placide,Pass,11,Gaëtan Charbonnier
4,5,6185,161,ShotOnPost,0.919,0.579,0.570862,OpenPlay,RightFoot,Marco Verratti,Pass,17,Zlatan Ibrahimovic


In [52]:
dfs.result.value_counts()

MissedShots    90582
BlockedShot    56385
SavedShot      53580
Goal           24075
ShotOnPost      4411
OwnGoal          726
Name: result, dtype: int64

## Question 1a

In [8]:
dfmerge1a = pd.merge(df, dfs, on = 'game_id', how = 'left')
dfmerge1a[dfmerge1a['shot_id'].isna()]

,Div,Date,Y,Team_Home,Team_Away,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,pH,pD,pA,home_xG,away_xG,home_team_id,away_team_id,game_id,pOver,pUnder,shot_id,team_id,result,x,y,xG,situation,shot_type,player_assisted,last_action,minute,player
129847,Ligue_1,2017-04-16,16,Bastia,Lyon,0,3,0,1,11,15,4,7,0.161632,0.221509,0.61686,0.0,3.0,162,178,4238,0.600484,0.399516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This game was abandoned due to fan invasion

In [9]:
df = df[df.game_id != 4238]

## Question 1b

In [101]:
df1b = dfmerge1a[['Y','xG','player_assisted','player']]
df1b_assist = df1b[['Y','xG','player_assisted']]
df1b_assist = df1b_assist.groupby(['player_assisted','Y']).sum().reset_index()
df1b_assist = df1b_assist.rename(columns = {'player_assisted' : 'player','xG' : 'xG_assist'})
df1b_goals = df1b[['Y','xG','player']]
df1b_goals = df1b_goals.groupby(['player','Y']).sum().reset_index()

df1b = pd.merge(df1b_assist, df1b_goals, on = ['Y','player'], how = 'outer')
df1b['Total_xG'] = df1b.xG_assist + df1b.xG
df1b = df1b[['player','Y','Total_xG']]
df1b.sort_values(by='Total_xG', ascending = False).head(10)

,player,Y,Total_xG
5921,Lionel Messi,14,53.503719
1919,Cristiano Ronaldo,14,52.909403
6161,Luis Suárez,15,49.109693
7583,Neymar,15,46.124972
1920,Cristiano Ronaldo,15,44.195410
5924,Lionel Messi,17,44.046686
5922,Lionel Messi,15,42.973189
8646,Robert Lewandowski,18,42.756660
5925,Lionel Messi,18,41.332334
5923,Lionel Messi,16,40.840305


## Question 1d

In [18]:
dfs.shot_type.value_counts()

RightFoot        118390
LeftFoot          72346
Head              38189
OtherBodyPart       834
Name: shot_type, dtype: int64

In [19]:
OG = dfs.loc[(dfs.result == 'OwnGoal')].groupby(['game_id','team_id']).result.count().rename('OG')
SOP = dfs.loc[(dfs.result == 'ShotOnPost')].groupby(['game_id','team_id']).result.count().rename('SOP')
HG = dfs.loc[(dfs.result == 'Goal')&(dfs.shot_type == 'Head')].groupby(['game_id','team_id']).result.count().rename('HG')
df_all = df.merge(OG, how = 'left', right_index = True, left_on = ['game_id','home_team_id'])
df_all = df_all.merge(OG, how = 'left', right_index = True, left_on = ['game_id','away_team_id'], suffixes = ['_Home','_Away'])
df_all = df_all.merge(SOP, how = 'left', right_index = True, left_on = ['game_id','home_team_id'])
df_all = df_all.merge(SOP, how = 'left', right_index = True, left_on = ['game_id','away_team_id'], suffixes = ['_Home','_Away'])
df_all = df_all.merge(HG, how = 'left', right_index = True, left_on = ['game_id','home_team_id'])
df_all = df_all.merge(HG, how = 'left', right_index = True, left_on = ['game_id','away_team_id'], suffixes = ['_Home','_Away'])
flds = ['OG_Home','OG_Away','SOP_Home','SOP_Away','HG_Home','HG_Away']
df_all.loc[:,flds] = df_all.loc[:,flds].fillna(0)
df_all.head()

,Div,Date,Y,Team_Home,Team_Away,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,pH,pD,pA,home_xG,away_xG,home_team_id,away_team_id,game_id,pOver,pUnder,OG_Home,OG_Away,SOP_Home,SOP_Away,HG_Home,HG_Away
5306,Ligue_1,2014-08-08,14,Reims,Paris SG,2,2,2,1,9,16,3,6,0.089841,0.196675,0.713484,1.367870,2.655380,177,161,6185,0.531328,0.468672,0.0,0.0,0.0,1.0,0.0,0.0
5307,Ligue_1,2014-08-09,14,Bastia,Marseille,3,3,1,2,13,9,4,4,0.245226,0.292215,0.462559,1.396890,1.317510,162,164,6186,0.433584,0.566416,1.0,0.0,0.0,0.0,0.0,1.0
5308,Ligue_1,2014-08-09,14,Evian Thonon Gaillard,Caen,0,3,0,3,10,12,2,7,0.399734,0.297363,0.302904,0.813737,1.238690,209,165,6187,0.409639,0.590361,0.0,0.0,0.0,0.0,0.0,1.0
5309,Ligue_1,2014-08-09,14,Guingamp,St Etienne,0,2,0,1,6,7,3,2,0.282701,0.312602,0.404697,0.632940,1.495320,169,175,6188,0.379147,0.620853,0.0,0.0,0.0,0.0,0.0,0.0
5310,Ligue_1,2014-08-09,14,Lille,Metz,0,0,0,0,14,2,3,1,0.636756,0.235893,0.127351,1.544680,0.057137,160,180,6189,0.428922,0.571078,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_all['OG_Total'] = df_all.OG_Home + df_all.OG_Away
df_all['SOP_Total'] = df_all.SOP_Home + df_all.SOP_Away
df_all['HG_Total'] = df_all.HG_Home + df_all.HG_Away

In [31]:
df_OG = df_all.loc[df_all.OG_Total == df_all.OG_Total.max()]
df_OG[['Div','Date','Y','Team_Home','Team_Away','OG_Home','OG_Away']]

,Div,Date,Y,Team_Home,Team_Away,OG_Home,OG_Away
76,EPL,2014-10-18,14,Southampton,Sunderland,0.0,3.0
77,EPL,2014-10-19,14,QPR,Liverpool,2.0,1.0
7516,Serie_A,2015-04-30,14,Empoli,Napoli,1.0,2.0


In [32]:
df_SOP = df_all.loc[df_all.SOP_Total == df_all.SOP_Total.max()]
df_SOP[['Div','Date','Y','Team_Home','Team_Away','SOP_Home','SOP_Away']]

,Div,Date,Y,Team_Home,Team_Away,SOP_Home,SOP_Away
1933,La_Liga,2014-09-21,14,Sociedad,Almeria,4.0,1.0
2470,La_Liga,2016-01-17,15,Getafe,Espanol,4.0,1.0
1123,EPL,2017-05-14,16,West Ham,Liverpool,2.0,3.0
3190,La_Liga,2017-12-17,17,Barcelona,La Coruna,5.0,0.0
5152,Bundesliga,2018-12-23,18,Hoffenheim,Mainz,4.0,1.0


In [33]:
df_HG = df_all.loc[df_all.HG_Total == df_all.HG_Total.max()]
df_HG[['Div','Date','Y','Team_Home','Team_Away','HG_Home','HG_Away']]

,Div,Date,Y,Team_Home,Team_Away,HG_Home,HG_Away
8030,Serie_A,2016-10-23,16,Cagliari,Fiorentina,3.0,2.0


In [35]:
df_all['GD_Home'] = df_all.FTHG - df_all.FTAG
df_all['SD_Home'] = df_all.HS - df_all.AS
df_all['xGD_Home'] = df_all.home_xG - df_all.away_xG
df_all['OG_Home'] = df_all.OG_Home - df_all.OG_Away
df_all['SOP_Home'] = df_all.SOP_Home - df_all.SOP_Away
df_all['HG_Home'] = df_all.HG_Home - df_all.HG_Away
flds = ['GD','SD','xGD','OG','SOP','HG']
for fld in flds:
    df_all[f'{fld}_Away'] = -df_all[f'{fld}_Home']
df_fil = df_all.loc[:,['Div','Date','Y','game_id','Team_Home','Team_Away','pH']+
                    [f'{fld}_{ha}' for fld in flds for ha in ['Home','Away']]]
df_fil.head()

,Div,Date,Y,game_id,Team_Home,Team_Away,pH,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,OG_Home,OG_Away,SOP_Home,SOP_Away,HG_Home,HG_Away
5306,Ligue_1,2014-08-08,14,6185,Reims,Paris SG,0.089841,0,0,-7,7,-1.287510,1.287510,0.0,-0.0,-2.0,2.0,0.0,-0.0
5307,Ligue_1,2014-08-09,14,6186,Bastia,Marseille,0.245226,0,0,4,-4,0.079380,-0.079380,2.0,-2.0,0.0,-0.0,-2.0,2.0
5308,Ligue_1,2014-08-09,14,6187,Evian Thonon Gaillard,Caen,0.399734,-3,3,-2,2,-0.424953,0.424953,0.0,-0.0,0.0,-0.0,-2.0,2.0
5309,Ligue_1,2014-08-09,14,6188,Guingamp,St Etienne,0.282701,-2,2,-1,1,-0.862380,0.862380,0.0,-0.0,0.0,-0.0,0.0,-0.0
5310,Ligue_1,2014-08-09,14,6189,Lille,Metz,0.636756,0,0,12,-12,1.487543,-1.487543,0.0,-0.0,0.0,-0.0,0.0,-0.0


In [36]:
df_long = pd.wide_to_long(df_fil, ['Team','GD','SD','xGD','OG','SOP','HG'], 'game_id','isHome','_',r'\w+').reset_index()
df_long.isHome = df_long.isHome == 'Home'
df_long = df_long.sort_values(['Date','game_id','isHome']).reset_index(drop = True)
df_long.head()

,game_id,isHome,pH,Date,Div,Y,Team,GD,SD,xGD,OG,SOP,HG
0,6185,False,0.089841,2014-08-08,Ligue_1,14,Paris SG,0,7,1.287510,-0.0,2.0,-0.0
1,6185,True,0.089841,2014-08-08,Ligue_1,14,Reims,0,-7,-1.287510,0.0,-2.0,0.0
2,6186,False,0.245226,2014-08-09,Ligue_1,14,Marseille,0,-4,-0.079380,-2.0,-0.0,2.0
3,6186,True,0.245226,2014-08-09,Ligue_1,14,Bastia,0,4,0.079380,2.0,0.0,-2.0
4,6187,False,0.399734,2014-08-09,Ligue_1,14,Caen,3,2,0.424953,-0.0,-0.0,2.0


In [37]:
gp = df_long.groupby(['Y','Team'])
df_long['sn'] = gp.Y.cumcount()
df_long['savgGD'] = gp.GD.transform(lambda x : x.cumsum().shift(1, fill_value = 0)) / (5+gp.Y.cumcount())
df_long['savgSD'] = gp.SD.transform(lambda x : x.cumsum().shift(1, fill_value = 0)) / (5+gp.Y.cumcount())
df_long['savgxGD'] = gp.xGD.transform(lambda x : x.cumsum().shift(1, fill_value = 0)) / (5+gp.Y.cumcount())
df_long['savgOG'] = gp.OG.transform(lambda x : x.cumsum().shift(1, fill_value = 0)) / (5+gp.Y.cumcount())
df_long['savgSOP'] = gp.SOP.transform(lambda x : x.cumsum().shift(1, fill_value = 0)) / (5+gp.Y.cumcount())
df_long['savgHG'] = gp.HG.transform(lambda x : x.cumsum().shift(1, fill_value = 0)) / (5+gp.Y.cumcount())
sflds = [f for f in df_long.columns if f.startswith('s')]

In [38]:
df_long.tail()

,game_id,isHome,pH,Date,Div,Y,Team,GD,SD,xGD,OG,SOP,HG,sn,savgGD,savgSD,savgxGD,savgOG,savgSOP,savgHG
18253,9954,True,0.249003,2019-05-26,Serie_A,18,Sampdoria,2,4,0.199758,0.0,0.0,0.0,37,0.166667,-0.357143,-0.030473,0.047619,-0.095238,0.000000
18254,9955,False,0.155177,2019-05-26,Serie_A,18,Milan,1,9,1.350616,-0.0,-2.0,-4.0,37,0.428571,1.404762,0.216964,-0.142857,-0.142857,0.047619
18255,9955,True,0.155177,2019-05-26,Serie_A,18,Spal,-1,-9,-1.350616,0.0,2.0,4.0,37,-0.261905,-1.452381,-0.223251,0.000000,-0.380952,0.190476
18256,9956,False,0.416710,2019-05-26,Serie_A,18,Lazio,-2,0,-1.218840,-0.0,-0.0,-0.0,37,0.285714,2.833333,0.381868,0.000000,0.333333,0.047619
18257,9956,True,0.416710,2019-05-26,Serie_A,18,Torino,2,0,1.218840,0.0,0.0,0.0,37,0.309524,0.857143,-0.087877,-0.095238,0.000000,0.047619


In [39]:
dfm = df_fil.merge(df_long.loc[df_long.isHome,['game_id']+sflds], how = 'inner', on = 'game_id')
dfm = dfm.merge(df_long.loc[~df_long.isHome,['game_id']+sflds], how = 'inner', on = 'game_id', suffixes = ['_Home','_Away'])
dfm.head()

,Div,Date,Y,game_id,Team_Home,Team_Away,pH,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,OG_Home,OG_Away,SOP_Home,SOP_Away,HG_Home,HG_Away,sn_Home,savgGD_Home,savgSD_Home,savgxGD_Home,savgOG_Home,savgSOP_Home,savgHG_Home,sn_Away,savgGD_Away,savgSD_Away,savgxGD_Away,savgOG_Away,savgSOP_Away,savgHG_Away
0,Ligue_1,2014-08-08,14,6185,Reims,Paris SG,0.089841,0,0,-7,7,-1.287510,1.287510,0.0,-0.0,-2.0,2.0,0.0,-0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,Ligue_1,2014-08-09,14,6186,Bastia,Marseille,0.245226,0,0,4,-4,0.079380,-0.079380,2.0,-2.0,0.0,-0.0,-2.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ligue_1,2014-08-09,14,6187,Evian Thonon Gaillard,Caen,0.399734,-3,3,-2,2,-0.424953,0.424953,0.0,-0.0,0.0,-0.0,-2.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ligue_1,2014-08-09,14,6188,Guingamp,St Etienne,0.282701,-2,2,-1,1,-0.862380,0.862380,0.0,-0.0,0.0,-0.0,0.0,-0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ligue_1,2014-08-09,14,6189,Lille,Metz,0.636756,0,0,12,-12,1.487543,-1.487543,0.0,-0.0,0.0,-0.0,0.0,-0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
dfm['GDD'] = dfm.savgGD_Home - dfm.savgGD_Away
dfm['xGDD'] = dfm.savgxGD_Home - dfm.savgxGD_Away
dfm['SDD'] = dfm.savgSD_Home - dfm.savgSD_Away
dfm['OGD'] = dfm.savgOG_Home - dfm.savgOG_Away
dfm['SOPD'] = dfm.savgSOP_Home - dfm.savgSOP_Away
dfm['HGD'] = dfm.savgHG_Home - dfm.savgHG_Away
df48 = dfm.loc[dfm.sn_Home+dfm.sn_Away > 9,['Div','Date','Y','game_id','Team_Home','Team_Away','pH']+[f'{f}D' for f in flds]]
df47 = df48.loc[df48['Y'] < 18]
df47.head()

,Div,Date,Y,game_id,Team_Home,Team_Away,pH,GDD,SDD,xGDD,OGD,SOPD,HGD
169,Ligue_1,2014-09-19,14,6155,Bordeaux,Evian Thonon Gaillard,0.629685,1.5,0.3,0.369822,-0.2,0.0,-0.4
188,Ligue_1,2014-09-20,14,6156,Marseille,Rennes,0.609827,0.2,5.2,0.610045,-0.2,0.4,0.2
189,Ligue_1,2014-09-20,14,6157,Lorient,Reims,0.553418,0.6,3.6,0.612688,0.4,0.6,-0.2
190,Ligue_1,2014-09-20,14,6158,Metz,Bastia,0.447615,-0.1,1.1,-0.111996,-0.2,0.0,0.4
191,Ligue_1,2014-09-20,14,6159,Nantes,Nice,0.530376,0.3,2.5,0.700922,0.0,0.4,-0.2


In [43]:
smf.ols('logit(pH) ~ OGD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     277.4
Date:                Sun, 21 Mar 2021   Prob (F-statistic):           5.38e-61
Time:                        14:27:56   Log-Likelihood:                -8342.2
No. Observations:                6321   AIC:                         1.669e+04
Df Residuals:                    6319   BIC:                         1.670e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2238      0.011    -19.644      0.000      -0.246      -0.201
OGD           -1.2714      0.076    -16.656      0.000      -1.421      -1.122
==============================================================================
Omnibus:                       44.160   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.529
Skew:                          -0.079   Prob(JB):                     4.36e-14
Kurtosis:                       3.457   Cond. No.                         6.70
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
smf.ols('logit(pH) ~ GDD + OGD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                 1.123e+04
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:20:26   Log-Likelihood:                -3686.5
No. Observations:                6321   AIC:                             7379.
Df Residuals:                    6318   BIC:                             7399.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2072      0.005    -37.969      0.000      -0.218      -0.196
GDD            0.9648      0.007    145.758      0.000       0.952       0.978
OGD            0.1739      0.038      4.592      0.000       0.100       0.248
==============================================================================
Omnibus:                       76.282   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              131.654
Skew:                          -0.049   Prob(JB):                     2.58e-29
Kurtosis:                       3.700   Cond. No.                         6.95
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [55]:
smf.ols('logit(pH) ~ OGD + xGDD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                 1.509e+04
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:20:39   Log-Likelihood:                -2935.0
No. Observations:                6321   AIC:                             5876.
Df Residuals:                    6318   BIC:                             5896.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2044      0.005    -42.184      0.000      -0.214      -0.195
OGD           -0.0589      0.033     -1.771      0.077      -0.124       0.006
xGDD           1.2263      0.007    169.248      0.000       1.212       1.240
==============================================================================
Omnibus:                      106.763   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              208.030
Skew:                          -0.054   Prob(JB):                     6.71e-46
Kurtosis:                       3.882   Cond. No.                         6.87
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
#Skill
smf.ols('logit(pH) ~ SOPD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     2298.
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:20:44   Log-Likelihood:                -7497.9
No. Observations:                6321   AIC:                         1.500e+04
Df Residuals:                    6319   BIC:                         1.501e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2167      0.010    -21.737      0.000      -0.236      -0.197
SOPD           1.1029      0.023     47.933      0.000       1.058       1.148
==============================================================================
Omnibus:                       21.274   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.939
Skew:                          -0.061   Prob(JB):                     2.33e-06
Kurtosis:                       3.289   Cond. No.                         2.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
smf.ols('logit(pH) ~ GDD+SOPD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                 1.235e+04
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:20:49   Log-Likelihood:                -3449.8
No. Observations:                6321   AIC:                             6906.
Df Residuals:                    6318   BIC:                             6926.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2060      0.005    -39.202      0.000      -0.216      -0.196
GDD            0.8855      0.007    128.159      0.000       0.872       0.899
SOPD           0.3088      0.014     22.676      0.000       0.282       0.336
==============================================================================
Omnibus:                       68.239   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.718
Skew:                          -0.026   Prob(JB):                     7.45e-26
Kurtosis:                       3.661   Cond. No.                         2.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
smf.ols('logit(pH) ~ xGDD+SOPD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                 1.555e+04
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:20:57   Log-Likelihood:                -2856.7
No. Observations:                6321   AIC:                             5719.
Df Residuals:                    6318   BIC:                             5740.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2040      0.005    -42.633      0.000      -0.213      -0.195
xGDD           1.1770      0.008    145.325      0.000       1.161       1.193
SOPD           0.1628      0.013     12.717      0.000       0.138       0.188
==============================================================================
Omnibus:                       97.027   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              183.664
Skew:                          -0.044   Prob(JB):                     1.31e-40
Kurtosis:                       3.831   Cond. No.                         2.86
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
smf.ols('logit(pH) ~ HGD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     1432.
Date:                Wed, 24 Mar 2021   Prob (F-statistic):          1.21e-282
Time:                        14:21:00   Log-Likelihood:                -7832.5
No. Observations:                6321   AIC:                         1.567e+04
Df Residuals:                    6319   BIC:                         1.568e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2182      0.011    -20.762      0.000      -0.239      -0.198
HGD            0.9820      0.026     37.842      0.000       0.931       1.033
==============================================================================
Omnibus:                       44.719   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.586
Skew:                          -0.093   Prob(JB):                     6.98e-14
Kurtosis:                       3.442   Cond. No.                         2.47
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
smf.ols('logit(pH) ~ HGD + GDD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                 1.120e+04
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:21:03   Log-Likelihood:                -3692.1
No. Observations:                6321   AIC:                             7390.
Df Residuals:                    6318   BIC:                             7411.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2072      0.005    -37.947      0.000      -0.218      -0.197
HGD           -0.0491      0.016     -3.143      0.002      -0.080      -0.018
GDD            0.9686      0.007    130.760      0.000       0.954       0.983
==============================================================================
Omnibus:                       68.560   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.888
Skew:                          -0.033   Prob(JB):                     6.84e-26
Kurtosis:                       3.660   Cond. No.                         2.96
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
smf.ols('logit(pH) ~ HGD + xGDD',df47).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logit(pH)   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                 1.519e+04
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:21:07   Log-Likelihood:                -2917.6
No. Observations:                6321   AIC:                             5841.
Df Residuals:                    6318   BIC:                             5862.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2042      0.005    -42.257      0.000      -0.214      -0.195
HGD            0.0818      0.013      6.160      0.000       0.056       0.108
xGDD           1.2077      0.008    153.625      0.000       1.192       1.223
==============================================================================
Omnibus:                      110.608   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              217.421
Skew:                          -0.060   Prob(JB):                     6.13e-48
Kurtosis:                       3.901   Cond. No.                         2.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""